In [2]:
%cd hypernet

/home/z1157095/hypernet-cnn/hypernet


In [3]:
from dotenv import load_dotenv
load_dotenv()

import random
import os

In [4]:
from comet_ml import Experiment, Optimizer

import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
import torch.utils.data as data_utils
import pandas as pd
from collections import defaultdict

torch.set_default_dtype(torch.float32)

In [5]:
from torchsummary import summary
import matplotlib.pyplot as plt
from tqdm import tqdm, trange

In [6]:
import tabular_hypernet as hp

In [7]:
hp.training_utils.get_dataset

<function tabular_hypernet.training_utils.get_dataset(size=60000, masked=False, mask_no=200, mask_size=700, shared_mask=False, batch_size=32, test_batch_size=32)>

In [8]:
os.environ['COMET_KEY'] = 'UXrV5UxyhTK3cyQNG6BDuc4bE'
os.environ.get("COMET_KEY")

'UXrV5UxyhTK3cyQNG6BDuc4bE'

In [9]:
# TODO

# TabSSLCrossEntropyLoss if confidence is high for both masks, add selfsl loss

# Verify if masks always divided into pairs the same way

In [10]:
class TabSSLCrossEntropyLoss(torch.nn.Module):
    def __init__(self, beta=0.1, unsup_target_wrapper=torch.nn.functional.softmax, threshold=None):
        super(TabSSLCrossEntropyLoss, self).__init__()
        
        self.y_f1 = torch.nn.CrossEntropyLoss()
        self.y_f2 = torch.nn.CrossEntropyLoss()
        
        self.f1_f2 = torch.nn.CrossEntropyLoss()
        self.f2_f1 = torch.nn.CrossEntropyLoss()
        
        self.beta = beta
        self.unsup_target_wrapper = unsup_target_wrapper
        self.threshold = threshold
        
    def is_observ_above_threshold(self, data):
        mask = torch.any(data >= self.threshold, dim=1)
        
        return mask
        
    
    def forward(self, sup_input, unsup_input):
        sup_outputs1, sup_outputs2, sup_labels = sup_input
        unsup_outputs1, unsup_outputs2 = unsup_input
        
        self.supervised_loss = self.y_f1(sup_outputs1, sup_labels) + self.y_f2(sup_outputs2, sup_labels)
        
        self.self_supervised_loss = 0
        if self.beta:

            if self.threshold:
                unsup_outputs1_target = torch.nn.functional.softmax(unsup_outputs1, dim=1)
                mask1 = self.is_observ_above_threshold(unsup_outputs1_target)

                if len(unsup_outputs1_target[mask1]):
                    unsup_outputs1_target = torch.argmax(unsup_outputs1_target[mask1], dim=1)
                    self.self_supervised_loss += self.f2_f1(unsup_outputs2[mask1], unsup_outputs1_target)

                unsup_outputs2_target = torch.nn.functional.softmax(unsup_outputs2, dim=1)
                mask2 = self.is_observ_above_threshold(unsup_outputs2_target)

                if len(unsup_outputs2_target[mask2]):
                    unsup_outputs2_target = torch.argmax(unsup_outputs2_target[mask2], dim=1)
                    self.self_supervised_loss += self.f1_f2(unsup_outputs1[mask2], unsup_outputs2_target)

            else:
                self.self_supervised_loss = self.f1_f2(unsup_outputs1, self.unsup_target_wrapper(unsup_outputs2, dim=1)) \
                                    + self.f2_f1(unsup_outputs2, self.unsup_target_wrapper(unsup_outputs1, dim=1))      
        
        return self.supervised_loss + self.beta * self.self_supervised_loss


### Setup for training

In [11]:
seed = 5

In [12]:
np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)

In [13]:
mask_size = 100

In [14]:
dataset = hp.semisl.get_train_test_sets()

In [15]:
epochs = 100

masks_no = 100


results = defaultdict(list)
size = (100, 59900)

In [16]:
#68.6   1.78

### Test shuffle masks

In [17]:
for beta in [.1, 1.]:
    for loss_threshold in [0.8]:
        for lr in [3e-5]:


            criterion = TabSSLCrossEntropyLoss(beta=beta, threshold=loss_threshold)

            np.random.seed(seed)
            torch.manual_seed(seed)
            random.seed(seed)

            # dataset


            hypernet = hp.Hypernetwork(
                architecture=torch.nn.Sequential(
                    torch.nn.Linear(784, 64), 
                    torch.nn.ReLU(),
                    torch.nn.Linear(64, 256),
                    torch.nn.ReLU(),
                    torch.nn.Linear(256, 128),
                ),
                target_architecture=[(mask_size, 100), (100, 10)],
                test_nodes=masks_no,
            ).cuda()
            

            hypernet = hypernet.train()



            optimizer = torch.optim.Adam(hypernet.parameters(), lr=lr)

            # loaders
            sup_trainloader, unsup_trainloader, testloader = hp.semisl.get_dataloaders(dataset=dataset, size=size, batch_size=32, test_batch_size=64)
            trainloader = hp.semisl.TrainDataLoaderSemi(sup_trainloader, unsup_trainloader)

            results[size].append(hp.semisl.train_semisl(hypernet,
                                              optimizer,
                                              criterion,
                                              (trainloader, testloader), 
                                              size,
                                              epochs,
                                              masks_no,
                                              changing_beta=None,
                                              log_to_comet=True,
                                              tags=['mask pairs shuffle'],
                                              description="""
                                              compare masks shuffled and not shuffled after list is over
                                              """,
                                            log_params={'seed': seed, 'temp scheduler': 'none', 'shuffled masks': 'true'}
                                            ))


torch.Size([1, 128])


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


!! log to comet is True 



COMET INFO: Experiment is live on comet.ml https://www.comet.com/abulenok/semi-hypernetwork/88791f41efb34bc5939a2953992361c2

100%|████████████████████████████████████████████████████████████████████████| 100/100 [38:09<00:00, 22.89s/it, loss=1.73, test_acc=75.8]
COMET ERROR: Error sending a notification, make sure you have opted-in for notifications
COMET INFO: Uploading metrics, params, and assets to Comet before program termination (may take several seconds)
COMET INFO: The Python SDK has 3600 seconds to finish before aborting...
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


torch.Size([1, 128])
!! log to comet is True 



COMET INFO: Experiment is live on comet.ml https://www.comet.com/abulenok/semi-hypernetwork/499b5e9e77244d139b3598eba68a4a3c

100%|████████████████████████████████████████████████████████████████████████| 100/100 [38:52<00:00, 23.32s/it, loss=1.74, test_acc=74.3]
COMET ERROR: Error sending a notification, make sure you have opted-in for notifications
COMET INFO: Uploading 1 metrics, params and output messages


In [18]:
for beta in [.1, 1.]:
    for loss_threshold in [0.8]:
        for lr in [3e-5]:


            criterion = TabSSLCrossEntropyLoss(beta=beta, threshold=loss_threshold)

            np.random.seed(seed)
            torch.manual_seed(seed)
            random.seed(seed)

            # dataset


            hypernet = hp.Hypernetwork(
                architecture=torch.nn.Sequential(
                    torch.nn.Linear(784, 64), 
                    torch.nn.ReLU(),
                    torch.nn.Linear(64, 256),
                    torch.nn.ReLU(),
                    torch.nn.Linear(256, 128),
                ),
                target_architecture=[(mask_size, 100), (100, 10)],
                test_nodes=masks_no,
            ).cuda()
            

            hypernet = hypernet.train()



            optimizer = torch.optim.Adam(hypernet.parameters(), lr=lr)

            # loaders
            sup_trainloader, unsup_trainloader, testloader = hp.semisl.get_dataloaders(dataset=dataset, size=size, batch_size=32, test_batch_size=64)
            trainloader = hp.semisl.TrainDataLoaderSemi(sup_trainloader, unsup_trainloader)

            results[size].append(hp.semisl.train_semisl(hypernet,
                                              optimizer,
                                              criterion,
                                              (trainloader, testloader), 
                                              size,
                                              epochs,
                                              masks_no,
                                              changing_beta=None,
                                              log_to_comet=True,
                                              tags=['mask pairs shuffle'],
                                              description="""
                                              compare masks shuffled and not shuffled after list is over
                                              """,
                                              shuffled_masks=False,
                                            log_params={'seed': seed, 'temp scheduler': 'none', 'shuffled masks': 'false'}
                                            ))


torch.Size([1, 128])
!! log to comet is True 



COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.com/abulenok/semi-hypernetwork/4c6c9323a3884a7abd9bd4f1c312470a

100%|████████████████████████████████████████████████████████████████████████| 100/100 [39:16<00:00, 23.57s/it, loss=1.73, test_acc=74.6]
COMET ERROR: Error sending a notification, make sure you have opted-in for notifications
COMET INFO: Uploading 1 metrics, params and output messages
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


torch.Size([1, 128])
!! log to comet is True 



COMET INFO: Experiment is live on comet.ml https://www.comet.com/abulenok/semi-hypernetwork/3dd65adced804bd1a4e47939117550b3

100%|████████████████████████████████████████████████████████████████████████| 100/100 [38:57<00:00, 23.38s/it, loss=1.76, test_acc=72.1]
COMET ERROR: Error sending a notification, make sure you have opted-in for notifications
COMET INFO: Uploading 1 metrics, params and output messages


##### Semi sl 

In [17]:
for beta in [1., 2.]:
    for loss_threshold in [0.2, 0.4]:
        for lr in [3e-5]:


            criterion = TabSSLCrossEntropyLoss(beta=beta, threshold=loss_threshold)

            np.random.seed(seed)
            torch.manual_seed(seed)
            random.seed(seed)

            # dataset


            hypernet = hp.Hypernetwork(
                architecture=torch.nn.Sequential(
                    torch.nn.Linear(784, 64), 
                    torch.nn.ReLU(),
                    torch.nn.Linear(64, 256),
                    torch.nn.ReLU(),
                    torch.nn.Linear(256, 128),
                ),
                target_architecture=[(mask_size, 100), (100, 10)],
                test_nodes=masks_no,
            ).cuda()
            

            hypernet = hypernet.train()



            optimizer = torch.optim.Adam(hypernet.parameters(), lr=lr)

            # loaders
            sup_trainloader, unsup_trainloader, testloader = hp.semisl.get_dataloaders(dataset=dataset, size=size, batch_size=32, test_batch_size=64)
            trainloader = hp.semisl.TrainDataLoaderSemi(sup_trainloader, unsup_trainloader)

            results[size].append(hp.semisl.train_semisl(hypernet,
                                              optimizer,
                                              criterion,
                                              (trainloader, testloader), 
                                              size,
                                              epochs,
                                              masks_no,
                                              changing_beta=None,
                                              log_to_comet=True,
                                              tags=['fix different mask pairs during training bigger nodeHiddenSize 100'],
                                              description="""
                                              before masks in training loop were selected sequentially without permuting
                                              now they are gonna be permuted when we will finish iterating over masks
                                              """,
                                            log_params={'seed': seed, 'temp scheduler': 'none'}
                                            ))


torch.Size([1, 128])
!! log to comet is True 



COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.com/abulenok/semi-hypernetwork/21f4c841798c4579bdd2023b1a3f7a64

100%|██████████████████████████████████████████████████████████████| 100/100 [45:25<00:00, 27.26s/it, loss=1.77, test_acc=73.4]
COMET ERROR: Error sending a notification, make sure you have opted-in for notifications
COMET INFO: Uploading 1 metrics, params and output messages


torch.Size([1, 128])
!! log to comet is True 



COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.com/abulenok/semi-hypernetwork/f102b0a009c44004ba15be053a60ac4d

100%|██████████████████████████████████████████████████████████████| 100/100 [45:41<00:00, 27.42s/it, loss=1.76, test_acc=73.6]
COMET ERROR: Error sending a notification, make sure you have opted-in for notifications
COMET INFO: Uploading 1 metrics, params and output messages


torch.Size([1, 128])
!! log to comet is True 



COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.com/abulenok/semi-hypernetwork/89624e76248c4ee69b0b81b06b00208f

100%|██████████████████████████████████████████████████████████████| 100/100 [59:16<00:00, 35.57s/it, loss=1.78, test_acc=71.8]
COMET ERROR: Error sending a notification, make sure you have opted-in for notifications
COMET INFO: Uploading 1 metrics, params and output messages


torch.Size([1, 128])
!! log to comet is True 



COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.com/abulenok/semi-hypernetwork/de229193a11e4980b321627806bd7add

100%|██████████████████████████████████████████████████████████████| 100/100 [44:54<00:00, 26.95s/it, loss=1.77, test_acc=72.4]
COMET ERROR: Error sending a notification, make sure you have opted-in for notifications
COMET INFO: Uploading 1 metrics, params and output messages


### Crazy hyperparams

In [19]:
for beta in [100.]:
    for loss_threshold in [0.9]:
        for lr in [3e-5]:


            criterion = TabSSLCrossEntropyLoss(beta=beta, threshold=loss_threshold)

            np.random.seed(seed)
            torch.manual_seed(seed)
            random.seed(seed)

            # dataset


            hypernet = hp.Hypernetwork(
                architecture=torch.nn.Sequential(
                    torch.nn.Linear(784, 64), 
                    torch.nn.ReLU(),
                    torch.nn.Linear(64, 256),
                    torch.nn.ReLU(),
                    torch.nn.Linear(256, 128),
                ),
                target_architecture=[(mask_size, 100), (100, 10)],
                test_nodes=masks_no,
            ).cuda()
            

            hypernet = hypernet.train()



            optimizer = torch.optim.Adam(hypernet.parameters(), lr=lr)

            # loaders
            sup_trainloader, unsup_trainloader, testloader = hp.semisl.get_dataloaders(dataset=dataset, size=size, batch_size=32, test_batch_size=64)
            trainloader = hp.semisl.TrainDataLoaderSemi(sup_trainloader, unsup_trainloader)

            results[size].append(hp.semisl.train_semisl(hypernet,
                                              optimizer,
                                              criterion,
                                              (trainloader, testloader), 
                                              size,
                                              epochs,
                                              masks_no,
                                              changing_beta=None,
                                              log_to_comet=True,
                                              tags=['fix different mask pairs during training bigger nodeHiddenSize 100'],
                                              description="""
                                              before masks in training loop were selected sequentially without permuting
                                              now they are gonna be permuted when we will finish iterating over masks
                                              """,
                                            log_params={'seed': seed, 'temp scheduler': 'none'}
                                            ))


torch.Size([1, 128])
!! log to comet is True 



COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.com/abulenok/semi-hypernetwork/792f6915d7f540e2b0ae60dcb91d39d3

 53%|█████████████████████████████████▉                              | 53/100 [25:40<22:45, 29.06s/it, loss=2.38, test_acc=9.8]


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3397, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_3832898/3161500947.py", line 38, in <cell line: 1>
    results[size].append(hp.semisl.train_semisl(hypernet,
  File "/home/z1157095/hypernet-cnn/hypernet/tabular_hypernet/semisl.py", line 247, in train_semisl
  File "/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/comet_ml/monkey_patching.py", line 312, in wrapper
    return_value = original(*args, **kwargs)
  File "/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/torch/_tensor.py", line 363, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph, inputs=inputs)
  File "/home/z1157095/anaconda3/envs/hyper-mnist/lib/python3.9/site-packages/torch/autograd/__init__.py", line 173, in backward
    Variable._execution_engine.run

##### Variance-based feature selection

In [57]:
for beta in [0.]:
    for threshold_var in [0.2, 0.4, 0.7, 1]:
        for lr in [3e-5]:
            mask_selector = VarianceWithThresholdMasksSelector(dataset, mask_size=mask_size, threshold=threshold_var)
            
            criterion = TabSSLCrossEntropyLoss(beta=beta, threshold=None)
            
            np.random.seed(seed)
            torch.manual_seed(seed)
            random.seed(seed)
            
            # dataset
            
            
            hypernet = hp.Hypernetwork(
                architecture=torch.nn.Sequential(
                    torch.nn.Linear(784, 64), 
                    torch.nn.ReLU(),
                    torch.nn.Linear(64, 256),
                    torch.nn.ReLU(),
                    torch.nn.Linear(256, 128),
                ),
                target_architecture=[(mask_size, 10), (10, 10)],
                test_nodes=masks_no,
            ).cuda()
            hypernet._create_mask = None
            hypernet.test_mask = mask_selector(hypernet, hypernet.test_nodes)

            hypernet = hypernet.train()
            
            
            
            optimizer = torch.optim.Adam(hypernet.parameters(), lr=lr)

            # loaders
            sup_trainloader, unsup_trainloader, testloader = hp.semisl.get_dataloaders(dataset=dataset, size=size, batch_size=32, test_batch_size=64)
            trainloader = hp.semisl.TrainDataLoaderSemi(sup_trainloader, unsup_trainloader)

            results[size].append(hp.semisl.train_semisl(hypernet,
                                              optimizer,
                                              criterion,
                                              (trainloader, testloader), 
                                              size,
                                              epochs,
                                              masks_no,
                                              changing_beta=None,
                                              log_to_comet=False,
                                              tags=['mask variance selection with threshold'],
                                              description="""
                                              masks are selected based on feature variance (above threshold) 
                                              """,
                                            log_params={'seed': seed, 'selector_threshold': threshold_var}
                                            ))
            

torch.Size([1, 128])
Hypernetwork(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=128, bias=True)
    (output_layer): Linear(in_features=128, out_features=320, bias=True)
  )
  (dropout): Dropout(p=0.5, inplace=False)
)
lol tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

  7%|█▏               | 7/100 [02:37<34:56, 22.55s/it, loss=1.91, test_acc=67.1]


KeyboardInterrupt: 

In [19]:
hypernet_with_selector = hypernet

In [ ]:
m1 = hypernet.test_mask[0]
m1

In [ ]:
torch.sum(m1)

#### No variance-based feature selection

In [18]:
for beta in [0.]:

    for lr in [3e-5]:


        criterion = TabSSLCrossEntropyLoss(beta=beta, threshold=None)

        np.random.seed(seed)
        torch.manual_seed(seed)
        random.seed(seed)

        # dataset


        hypernet = hp.Hypernetwork(
            architecture=torch.nn.Sequential(
                torch.nn.Linear(784, 64), 
                torch.nn.ReLU(),
                torch.nn.Linear(64, 256),
                torch.nn.ReLU(),
                torch.nn.Linear(256, 128),
            ),
            target_architecture=[(mask_size, 10), (10, 10)],
            test_nodes=masks_no,
        ).cuda()
        
        hypernet = hypernet.train()



        optimizer = torch.optim.Adam(hypernet.parameters(), lr=lr)

        # loaders
        sup_trainloader, unsup_trainloader, testloader = hp.semisl.get_dataloaders(dataset=dataset, size=size, batch_size=32, test_batch_size=64)
        trainloader = hp.semisl.TrainDataLoaderSemi(sup_trainloader, unsup_trainloader)

        results[size].append(hp.semisl.train_semisl(hypernet,
                                          optimizer,
                                          criterion,
                                          (trainloader, testloader), 
                                          size,
                                          epochs,
                                          masks_no,
                                          changing_beta=None,
                                          log_to_comet=True,
                                          tags=['mask variance selection softmax temp increment'],
                                          description="""
                                          masks are selected based on feature variance with softmax and temp
                                          """,
                                        log_params={'seed': seed, 'temp scheduler': 'none: baseline'}
                                        ))


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


torch.Size([1, 128])


COMET INFO: Experiment is live on comet.ml https://www.comet.com/abulenok/semi-hypernetwork/a5a1e7d1c6a444a8a6f3a7875b202f7e

100%|████████████████| 100/100 [31:51<00:00, 19.11s/it, loss=1.8, test_acc=67.1]
COMET ERROR: Error sending a notification, make sure you have opted-in for notifications
COMET INFO: Uploading metrics, params, and assets to Comet before program termination (may take several seconds)
COMET INFO: The Python SDK has 3600 seconds to finish before aborting...


In [ ]:
m2 = hypernet.test_mask[0]
m2

In [ ]:
m1==m2

In [ ]:
torch.sum(m2)